### Part 1. Data Acquisition & Preprocessing
#### Staring with incorperating the dataset

In [35]:
%load_ext cudf.pandas
import torch
import pandas as pd

from cuml.manifold import UMAP
from cuml.cluster import hdbscan
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold


In [24]:
#Use for NVIDIA CUDA acceleration
torch_device = 'cpu'
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
    torch_device = 'cuda'
else:
    print("No GPU available.")

GPU: NVIDIA GeForce RTX 4080 Laptop GPU is available.


In [25]:
%pip install kaggle

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [36]:
# Downloading the dataset from Kaggle
!export KAGGLE_USERNAME=orranereid
#!kaggle datasets download -d elvinrustam/books-dataset --unzip --force

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Preproccessing Data Steps.

In [37]:
# Extracting the info from the CSV fiel
gb_md = pd.read_csv('BooksDataset.csv')

In [38]:
# Here we are printing the original data from the csv
gb_md

,Title,Authors,Description,Category,Publisher,Publish Date,Price
0,Goat Brothers,"By Colton, Larry",NaN,"History , General",Doubleday,"Friday, January 1, 1993",Price Starting at $8.79
1,The Missing Person,"By Grumbach, Doris",NaN,"Fiction , General",Putnam Pub Group,"Sunday, March 1, 1981",Price Starting at $4.99
2,Don't Eat Your Heart Out Cookbook,"By Piscatella, Joseph C.",NaN,"Cooking , Reference",Workman Pub Co,"Thursday, September 1, 1983",Price Starting at $4.99
3,When Your Corporate Umbrella Begins to Leak: A...,"By Davis, Paul D.",NaN,NaN,Natl Pr Books,"Monday, April 1, 1991",Price Starting at $4.99
4,Amy Spangler's Breastfeeding : A Parent's Guide,"By Spangler, Amy",NaN,NaN,Amy Spangler,"Saturday, February 1, 1997",Price Starting at $5.32
...,...,...,...,...,...,...,...
103077,Build 3 Super Serving Carts,By Chuck Hampton,NaN,NaN,ENDesigns Inc.,"Wednesday, January 1, 1992",Price Starting at $9.97
103078,My Land of Israel,"By Nover, Elizabeth Z.",NaN,"Juvenile Nonfiction , People & Places , Middl...",Behrman House,"Friday, May 1, 1987",Price Starting at $4.99
103079,Tongues: To Speak or Not to Speak,By Donald W. Burdick,NaN,NaN,Moody Press,"Wednesday, January 1, 1969",Price Starting at $5.29
103080,If I'm in charge here why is everybody laughing?,"By Campbell, David P.",NaN,NaN,Argus Communications,"Tuesday, January 1, 1980",Price Starting at $4.99


In [39]:
# Dropping all observations with missing values
gb_md = gb_md.dropna()
# Printing the length to see how much data we have left with complete info.
print(len(gb_md))
# Now we will be dropping the columns that are irrelevant to us
gb_md = gb_md.drop(columns=['Price', 'Publisher'])
#

65305


In [40]:
gb_md

,Title,Authors,Description,Category,Publish Date
7,Journey Through Heartsongs,"By Stepanek, Mattie J. T.",Collects poems written by the eleven-year-old ...,"Poetry , General","Saturday, September 1, 2001"
8,In Search of Melancholy Baby,"By Aksyonov, Vassily, Heim, Michael Henry, and...",The Russian author offers an affectionate chro...,"Biography & Autobiography , General","Monday, June 1, 1987"
10,The Dieter's Guide to Weight Loss During Sex,"By Smith, Richard","A humor classic, this tongue-in-cheek diet pla...","Health & Fitness , Diet & Nutrition , Diets","Sunday, January 1, 1978"
11,Germs : Biological Weapons and America's Secre...,"By Miller, Judith, Engelberg, Stephen, and Bro...","Deadly germs sprayed in shopping malls, bomb-l...","Technology & Engineering , Military Science","Monday, October 1, 2001"
13,The Good Book: Reading the Bible with Mind and...,"By Gomes, Peter J.","""The Bible and the social and moral consequenc...","Religion , Biblical Biography , General","Friday, May 1, 1998"
...,...,...,...,...,...
103069,Like A Sister: A Novel,"By Daugharty, Janice",Sister cannot say exactly when or where she wa...,"Fiction , Literary","Monday, November 1, 1999"
103071,Creating Web Pages Simplified (3-D Visual Series),"By Maran, Ruth, Whitehead, Paul, and Marangrap...","An ""owner's manual"" for first-time Web page cr...","Computers , Internet , General","Wednesday, January 1, 1997"
103072,EVA: The Real Key to Creating Wealth,"By Ehrbar, Al","Called ""today's hottest financial idea and get...","Business & Economics , Corporate Finance , Ge...","Thursday, October 1, 1998"
103075,The Essentials of Spanish (REA's Language Seri...,"By Mouat, Ricardo Gutierrez",REA’s Essentials provide quick and easy access...,"Foreign Language Study , Spanish","Thursday, January 1, 1998"


In [41]:
from transformers import AutoTokenizer, AutoModel
# sample GPT answer

# Load a pretrained model and tokenizer
model_name = "nomic-ai/nomic-embed-text-v1.5"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, device=torch_device)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, device=torch_device)

def embedding_gen(text):
    inputs=tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings


<All keys matched successfully>


In [42]:
arr = [str(i) for i in gb_md.iloc[0].drop('Description')]
print(arr)

['Journey Through Heartsongs', 'By Stepanek, Mattie J. T.', ' Poetry , General', 'Saturday, September 1, 2001']


In [ ]:
#sample_t1 = embedding_gen(gb_md.iloc[0])
#sample_t2 = embedding_gen(arr)
#sample_t = torch.cat((sample_t1, sample_t2))
results = [embedding_gen(i).numpy() for i in gb_md]

AttributeError: This 'Pipeline' has no attribute 'fit_predict'